<a href="https://colab.research.google.com/github/alex-jk/GTA_apartment_rentals/blob/main/code/gta_apartment_rentals_get_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade gspread google-auth google-auth-oauthlib google-auth-httplib2
!pip install requests
!pip install googlemaps

  Using cached googlemaps-4.10.0.tar.gz (33 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40711 sha256=83bfc8b328f3c42582539af8541abb29a51067365af85b98b104ab3c343eefad
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [2]:
import pandas as pd
import requests
import re
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [4]:
from google.colab import auth
auth.authenticate_user()

import google.auth
from google.colab import drive
drive.mount('/content/drive')

credentials, project = google.auth.default(scopes=['https://www.googleapis.com/auth/spreadsheets'])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Import input data GTA_apartment_buildings_list

In [5]:
import gspread

gc = gspread.authorize(credentials)

sheet_name = 'GTA_apartment_buildings_list'
sh = gc.open(sheet_name)
worksheet = sh.sheet1

# Get all the records of the data
apartments_data = worksheet.get_all_records()

# Convert to a DataFrame
apartments_data_df = pd.DataFrame(apartments_data)

# Display the first few rows of the dataframe
print(f"\n------------ Apartments data df shape: {apartments_data_df.shape}")
print("\n------------")
print(apartments_data_df.head())


------------ Apartments data df shape: (121, 7)

------------
                          apartment_address num_beds  \
0    155 Antibes Dr, North York, ON M2R 3G7  1, 2, 3   
1    300 Antibes Dr, North York, ON M2R 3N8  1, 2, 3   
2        901 Avenue Rd, Toronto, ON M5P 2K7  1, 2, 3   
3   155 Balliol Street, Toronto, ON M4S 1C4  1, 2, 3   
4  3575 Bathurst St, North York, ON M6A 2C8     1, 2   

                                       phone_numbers    Elementary_School  \
0                         437-973-3626, 416-665-4430          Rockford PS   
1           437-900-6800, 416-665-2100, 437-900-6811          Rockford PS   
2                         416-485-7207, 416-751-3368    Oriole Park Jr PS   
3           888-244-6696, 647-507-4461, 416-487-4461     Davisville Jr PS   
4  647-559-7317, 647-694-8048, 437-227-9562, 647-...  Ledbury Park E & MS   

         Middle_School           High_School street_name  
0    Fisherville Sr PS  Northview Heights SS     Antibes  
1    Fisherville Sr

In [6]:
def find_phone_number(address, query_str):
    query = address.replace(' ', '+') + query_str
    url = f"https://www.google.com/search?q={query}"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Use regular expressions to find phone numbers
        phone_regex = re.compile(r'\(\d{3}\)\s*\d{3}-\d{4}|\d{3}-\d{3}-\d{4}')
        text = soup.get_text()
        # print("\n----------------")
        # print(text)
        phone_numbers = phone_regex.findall(text)

        if phone_numbers:
            # Remove duplicates and format phone numbers consistently
            formatted_numbers = [re.sub(r'[^0-9]', '', number) for number in set(phone_numbers)]
            formatted_numbers = [f"{num[:3]}-{num[3:6]}-{num[6:]}" for num in formatted_numbers]
            formatted_numbers = list(set(formatted_numbers))

            return formatted_numbers
        else:
            return []
    else:
        return []

In [7]:
def expand_ranges(beds_list):
    expanded_list = []
    for item in beds_list:
        if '-' in item:  # Check if item is a range
            start, end = map(int, item.split('-'))
            expanded_list.extend(range(start, end + 1))  # Add all numbers in the range
        else:
            expanded_list.append(int(item))  # Add individual numbers directly
    return expanded_list

def find_num_beds(address, query_str):
    query = address.replace(' ', '+') + query_str
    query_apartments_com = address.replace(' ', '+') + "+apartments+com"
    url = f"https://www.google.com/search?q={query}"
    url_apartments_com = f"https://www.google.com/search?q={query_apartments_com}"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    text_numbers_to_int = {'one': 1, '1': 1, 'two': 2, '2': 2, 'three': 3, '3': 3, 'four': 4, 'five': 5}

    response = requests.get(url, headers=headers)
    response_apartments_com = requests.get(url_apartments_com, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Use regular expressions to find phone numbers
        numeric_bed_regex = re.compile(r'\b(\d+(?:-\d+)?)\s*beds?\b', re.IGNORECASE)
        text_bed_regex = re.compile(r'\b(one|two|three|four|five|1|2|3)\s*bedroom?\b', re.IGNORECASE)
        text = soup.get_text()
        # print("\n----------------")
        # print(text)

        numeric_beds_list = numeric_bed_regex.findall(text)
        expanded_beds_list = list(set( expand_ranges(numeric_beds_list) ))
        expanded_beds_list.sort()
        # print(f"\n numeric beds: {expanded_beds_list}")

        text_beds_list = text_bed_regex.findall(text)
        text_beds_list = list(set(text_beds_list))
        text_beds_list = [text_numbers_to_int[bed.lower()] for bed in text_beds_list]
        if len(text_beds_list) > 0:
          text_beds_list = list(range(1, max(text_beds_list) + 1))
        # print(f"\n text beds: {text_beds_list}")

        combined_set = set(expanded_beds_list + text_beds_list)
        combined_set = sorted(list(combined_set))

        # if response_apartments_com.status_code == 200:
        #   soup_apartments_com = BeautifulSoup(response_apartments_com.text, 'html.parser')
        #   links = soup.find_all('a')
        #   # Attempt to find the first valid apartments.com URL
        #   apartments_url = None
        #   for link in links:
        #       href = link.get('href')
        #       if 'apartments.com' in href:
        #           # Typical Google search result URLs are redirected, so parse them to get the actual URL
        #           start = href.find('http')
        #           end = href.find('&', start)
        #           if start != -1 and end != -1:
        #               apartments_url = href[start:end]
        #               break

        #   if apartments_url:
        #       print("Found Apartments.com URL:", apartments_url)
        #   else:
        #       print("No Apartments.com URL found in the first few search results.")

        if combined_set:
            smallest_bed_number = combined_set[0]
            complete_beds_list = list(range(1, smallest_bed_number)) + combined_set

            # Remove duplicates and sort again if necessary
            complete_beds_list = sorted(set(complete_beds_list))
            complete_beds_str = ', '.join(map(str, complete_beds_list))
            return complete_beds_str
        else:
            return []
    else:
        return []

In [8]:
# def conditionally_find_num_beds(row, query_str):
#     # If num_beds is missing, then call find_num_beds
#     if pd.isna(row['num_beds']) or not str(row['num_beds']).strip():
#         return find_num_beds(row['apartment_address'], query_str)
#     else:
#         return row['num_beds']

def conditionally_find_value(row, column_name, search_function, *args, **kwargs):
    """
    Checks if the specified column in a DataFrame row is missing or empty. If so, it calls
    a specified search function to find the value.

    Parameters:
    - row: The row of the DataFrame.
    - column_name: The name of the column to check and potentially fill.
    - search_function: The function to call if the column value is missing or empty.
                       This function should accept the apartment address as its first argument,
                       followed by any *args and **kwargs necessary for the search.
    - *args: Additional positional arguments to pass to the search_function.
    - **kwargs: Additional keyword arguments to pass to the search_function.

    Returns:
    - The original value from the row[column_name] if it exists and is not empty,
      otherwise the result from search_function.
    """
    # Check if the column value is missing or empty
    if pd.isna(row[column_name]) or not str(row[column_name]).strip():
        # Call the search function with the address, args, and kwargs
        return search_function(row['apartment_address'], *args, **kwargs)
    else:
        # Return the original value
        return row[column_name]

In [9]:
ind = 0

print(apartments_data_df['apartment_address'][ind])
print( find_phone_number(apartments_data_df['apartment_address'][ind], "+apartment+building+phone+number") )

print( find_num_beds(apartments_data_df['apartment_address'][ind], "+apartment+rent+beds") )

155 Antibes Dr, North York, ON M2R 3G7
['437-973-3626', '416-665-4430']
1, 2, 3


In [10]:
# Apply the find_phone_numbers function to each row
query_str = "+apartment+building+phone+number"

tqdm.pandas()
# apartments_data_df['phone_numbers'] = apartments_data_df['apartment_address'].progress_apply(lambda address: find_phone_number(address, query_str))
apartments_data_df['phone_numbers'] = apartments_data_df.progress_apply(conditionally_find_value, axis=1,
                                                                        column_name='phone_numbers',
                                                                        search_function=find_phone_number,
                                                                        query_str=query_str)

print(apartments_data_df.head(10))

100%|██████████| 121/121 [00:05<00:00, 22.82it/s]

                          apartment_address num_beds  \
0    155 Antibes Dr, North York, ON M2R 3G7  1, 2, 3   
1    300 Antibes Dr, North York, ON M2R 3N8  1, 2, 3   
2        901 Avenue Rd, Toronto, ON M5P 2K7  1, 2, 3   
3   155 Balliol Street, Toronto, ON M4S 1C4  1, 2, 3   
4  3575 Bathurst St, North York, ON M6A 2C8     1, 2   
5  3892 Bathurst St, North York, ON M3H 3N5     1, 2   
6  3905 Bathurst St, North York, ON M3H 3N7  1, 2, 3   
7  4222 Bathurst St, North York, ON M3H 3R2     1, 2   
8  4340 Bathurst St, North York, ON M3H 2M9     1, 2   
9  4383 Bathurst St, North York, ON M3H 3P8  1, 2, 3   

                                       phone_numbers  \
0                         437-973-3626, 416-665-4430   
1           437-900-6800, 416-665-2100, 437-900-6811   
2                         416-485-7207, 416-751-3368   
3           888-244-6696, 647-507-4461, 416-487-4461   
4  647-559-7317, 647-694-8048, 437-227-9562, 647-...   
5                                       647-559

In [11]:
# Apply the find_phone_numbers function to each row
query_str = "+apartment+rent+beds"

# apartments_data_df['num_beds'] = apartments_data_df.progress_apply(lambda row: conditionally_find_num_beds(row, query_str), axis=1)
apartments_data_df['num_beds'] = apartments_data_df.progress_apply(conditionally_find_value, axis=1,
                                                                        column_name='num_beds',
                                                                        search_function=find_num_beds,
                                                                        query_str=query_str)

print(apartments_data_df.head(10))

100%|██████████| 121/121 [00:13<00:00,  8.83it/s]

                          apartment_address num_beds  \
0    155 Antibes Dr, North York, ON M2R 3G7  1, 2, 3   
1    300 Antibes Dr, North York, ON M2R 3N8  1, 2, 3   
2        901 Avenue Rd, Toronto, ON M5P 2K7  1, 2, 3   
3   155 Balliol Street, Toronto, ON M4S 1C4  1, 2, 3   
4  3575 Bathurst St, North York, ON M6A 2C8     1, 2   
5  3892 Bathurst St, North York, ON M3H 3N5     1, 2   
6  3905 Bathurst St, North York, ON M3H 3N7  1, 2, 3   
7  4222 Bathurst St, North York, ON M3H 3R2     1, 2   
8  4340 Bathurst St, North York, ON M3H 2M9     1, 2   
9  4383 Bathurst St, North York, ON M3H 3P8  1, 2, 3   

                                       phone_numbers  \
0                         437-973-3626, 416-665-4430   
1           437-900-6800, 416-665-2100, 437-900-6811   
2                         416-485-7207, 416-751-3368   
3           888-244-6696, 647-507-4461, 416-487-4461   
4  647-559-7317, 647-694-8048, 437-227-9562, 647-...   
5                                       647-559

In [15]:
print(apartments_data_df.head(10))

                          apartment_address num_beds  \
0    155 Antibes Dr, North York, ON M2R 3G7  1, 2, 3   
1    300 Antibes Dr, North York, ON M2R 3N8  1, 2, 3   
2        901 Avenue Rd, Toronto, ON M5P 2K7  1, 2, 3   
3   155 Balliol Street, Toronto, ON M4S 1C4  1, 2, 3   
4  3575 Bathurst St, North York, ON M6A 2C8     1, 2   
5  3892 Bathurst St, North York, ON M3H 3N5     1, 2   
6  3905 Bathurst St, North York, ON M3H 3N7  1, 2, 3   
7  4222 Bathurst St, North York, ON M3H 3R2     1, 2   
8  4340 Bathurst St, North York, ON M3H 2M9     1, 2   
9  4383 Bathurst St, North York, ON M3H 3P8  1, 2, 3   

                                       phone_numbers  \
0                         437-973-3626, 416-665-4430   
1           437-900-6800, 416-665-2100, 437-900-6811   
2                         416-485-7207, 416-751-3368   
3           888-244-6696, 647-507-4461, 416-487-4461   
4  647-559-7317, 647-694-8048, 437-227-9562, 647-...   
5                                       647-559

In [16]:
def extract_street_name(address):
    match = re.search(r'\d+\s([A-Za-z\s]+)\s', address)
    if match:
        return match.group(1).strip()
    else:
        return None

ind = 7
print( apartments_data_df['apartment_address'].iloc[ind] )

extract_street_name( apartments_data_df['apartment_address'].iloc[ind] )

apartments_data_df['street_name'] = apartments_data_df['apartment_address'].apply(extract_street_name)

4222 Bathurst St, North York, ON M3H 3R2


In [17]:
apartments_data_df['num_beds'] = apartments_data_df['num_beds'].astype(str)

apartments_data_df.sort_values(by=['street_name', 'apartment_address'], inplace=True)
apartments_data_df.reset_index(drop=True, inplace=True)

In [18]:
from gspread_dataframe import set_with_dataframe

spreadsheet_title = 'GTA_apartment_buildings_output'

try:
    # Attempt to open the existing spreadsheet
    sh = gc.open(spreadsheet_title)
    # If successful, delete the spreadsheet by moving it to trash
    gc.del_spreadsheet(sh.id)
    print(f"Existing spreadsheet '{spreadsheet_title}' moved to trash.")
except gspread.SpreadsheetNotFound:
    # If the spreadsheet does not exist, just continue
    print(f"No existing spreadsheet named '{spreadsheet_title}' found. Creating a new one.")

sh = gc.create(spreadsheet_title)
worksheet = sh.get_worksheet(0)

set_with_dataframe(worksheet, apartments_data_df)

Existing spreadsheet 'GTA_apartment_buildings_output' moved to trash.


#### Import output dataset

In [19]:
sheet_name = 'GTA_apartment_buildings_output'
sh = gc.open(sheet_name)
worksheet = sh.sheet1

# Get all the records of the data
apartments_data_out = worksheet.get_all_records()

# Convert to a DataFrame
apartments_data_out_df = pd.DataFrame(apartments_data_out)

# create address column with no postal code
regex_pattern = r'\b[A-Z]\d[A-Z] \d[A-Z]\d\b'
apartments_data_out_df['address_no_postal_code'] = apartments_data_out_df['apartment_address'].str.replace(regex_pattern, '', regex=True).str.strip()

# Display the first few rows of the dataframe
print(f"\n------------ Apartments data df shape: {apartments_data_out_df.shape}")
print("\n------------")
print(apartments_data_out_df.head())


------------ Apartments data df shape: (121, 8)

------------
                         apartment_address num_beds  \
0   155 Antibes Dr, North York, ON M2R 3G7  1, 2, 3   
1   300 Antibes Dr, North York, ON M2R 3N8  1, 2, 3   
2       901 Avenue Rd, Toronto, ON M5P 2K7  1, 2, 3   
3  155 Balliol Street, Toronto, ON M4S 1C4  1, 2, 3   
4    2100 Bathurst St, Toronto, ON M5N 2P2     1, 2   

                              phone_numbers       Elementary_School  \
0                437-973-3626, 416-665-4430             Rockford PS   
1  437-900-6800, 416-665-2100, 437-900-6811             Rockford PS   
2                416-485-7207, 416-751-3368       Oriole Park Jr PS   
3  888-244-6696, 647-507-4461, 416-487-4461        Davisville Jr PS   
4                              416-781-0408  West Preparatory Jr PS   

            Middle_School           High_School street_name  \
0       Fisherville Sr PS  Northview Heights SS     Antibes   
1       Fisherville Sr PS  Northview Heights SS     A

In [20]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

geolocator = Nominatim(user_agent="myApp")

In [21]:
ind = 2
address_check = apartments_data_out_df['address_no_postal_code'].iloc[ind]
print(address_check)

location = geolocator.geocode(address_check)
lat = location.latitude
lon = location.longitude
print(lat, lon)

901 Avenue Rd, Toronto, ON
43.700344666666666 -79.40666533333334


In [22]:
print( geolocator.geocode('110 Inverlochy Blvd, Markham, ON') )

110, Inverlochy Boulevard, Langstaff, Markham, York Region, Golden Horseshoe, Ontario, L3T 3C0, Canada


In [23]:
def apply_geocode(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return (location.latitude, location.longitude)
        else:
            return (None, None)
    except Exception as e:
        print(f"Error geocoding address: {address} - {e}")
        return (None, None)

In [24]:
tqdm.pandas(desc="Geocoding addresses")
apartments_data_out_df['lat_long'] = apartments_data_out_df['address_no_postal_code'].progress_apply(apply_geocode)

print(apartments_data_out_df.head())

Geocoding addresses: 100%|██████████| 121/121 [01:00<00:00,  2.00it/s]

                         apartment_address num_beds  \
0   155 Antibes Dr, North York, ON M2R 3G7  1, 2, 3   
1   300 Antibes Dr, North York, ON M2R 3N8  1, 2, 3   
2       901 Avenue Rd, Toronto, ON M5P 2K7  1, 2, 3   
3  155 Balliol Street, Toronto, ON M4S 1C4  1, 2, 3   
4    2100 Bathurst St, Toronto, ON M5N 2P2     1, 2   

                              phone_numbers       Elementary_School  \
0                437-973-3626, 416-665-4430             Rockford PS   
1  437-900-6800, 416-665-2100, 437-900-6811             Rockford PS   
2                416-485-7207, 416-751-3368       Oriole Park Jr PS   
3  888-244-6696, 647-507-4461, 416-487-4461        Davisville Jr PS   
4                              416-781-0408  West Preparatory Jr PS   

            Middle_School           High_School street_name  \
0       Fisherville Sr PS  Northview Heights SS     Antibes   
1       Fisherville Sr PS  Northview Heights SS     Antibes   
2              Hodgson MS      North Toronto CI      

##### Create a map

In [25]:
import folium

# Create a base map
m = folium.Map(location=[43.75491558522391, -79.4229619534175], zoom_start=10)

# Iterate through the DataFrame and add each lat/long as a marker on the map
for index, row in apartments_data_out_df.iterrows():
    lat, lon = row['lat_long']
    phone_number = row['phone_numbers']
    apartment_address = row['apartment_address']
    beds = row['num_beds']
    elem_school = row['Elementary_School']
    middle_school = row['Middle_School']
    high_school = row['High_School']

    # Create a popup with the desired information
    # popup_text = f"Address: {apartment_address}<br>Bedrooms: {beds}<br>Phone: {phone_number}"
    popup_html = f"""
        <span style="font-size: 14px;"><strong>Address:</strong> {apartment_address}</span><br>
        <span style="font-size: 14px;"><strong>Bedrooms:</strong> {beds}</span><br>
        <span style="font-size: 14px;"><strong>Phone numbers:</strong> {phone_number}</span><br><br>
        <span style="font-size: 14px;"><strong>Elementary School:</strong> {elem_school}</span><br>
        <span style="font-size: 14px;"><strong>Middle School:</strong> {middle_school}</span><br>
        <span style="font-size: 14px;"><strong>High School:</strong> {high_school}</span>
        """
    popup = folium.Popup(popup_html, max_width=300)

    # folium.Marker([lat, lon]).add_to(m)
    folium.Marker([lat, lon], popup=popup).add_to(m)

# Save the map to an HTML file
m.save('my_map.html')

#### Get school addresses

In [26]:
def find_school_address(address, query_str):
    query = address.replace(' ', '+') + query_str

    url = f"https://www.google.com/search?q={query}"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        text = soup.get_text()
        print("\n----------------")
        print(text)

In [27]:
query_str = "+school+address+canada"

ind = 0

print( apartments_data_out_df['Elementary_School'].iloc[ind] )
find_school_address( apartments_data_out_df['Elementary_School'].iloc[ind], query_str )

Rockford PS

----------------
Rockford PS school address canada - Google Search×Please click here if you are not redirected within a few seconds.    AllMapsNewsImages Videos Shopping Books Search tools    Any timeAny timePast hourPast 24 hoursPast weekPast monthPast yearAll resultsAll resultsVerbatimRockford Public School / Address60 Rockford Rd, Toronto, ON M2R 3A7, CanadaRockford Public School - Toronto District School Boardwww.tdsb.on.ca › find-your › schools › schnoPhone:(416) 395-2820 ; Fax:(416) 395-4467 ; Address:60 Rockford Rd, North York, ON, M2R 3A7.People also askHow long is a school day in Canada?What age is elementary school in Canada?What grade is middle school in Canada?How many school aged children are there in Canada?Rockford Public School, 60 Rockford Rd, North York, ON - MapQuestwww.mapquest.com › Canada › Ontario › North YorkSee reviews, map, get the address, and find directions ... 60 Rockford Rd. North York, ON M2R 3A7. Opens at 8 ... Also at this address. Rockfor

In [31]:
import googlemaps

gmaps = googlemaps.Client(key='xxx')

place_result = gmaps.places(query=apartments_data_out_df['Elementary_School'].iloc[ind], type="school")

if place_result['status'] == 'OK':
    # Extract the formatted address of the first result
    formatted_address = place_result['results'][0]['formatted_address']
    print("Formatted Address:", formatted_address)
else:
    print("No results found or error in the request")

Formatted Address: 60 Rockford Rd, Toronto, ON M2R 3A7, Canada


In [ ]:
def get_address_from_google_places(query, place_type="school"):
    place_result = gmaps.places(query=query, type=place_type)

    if place_result['status'] == 'OK' and place_result['results']:
        return place_result['results'][0]['formatted_address']
    else:
        return "No results found or error in the request"

get_address_from_google_places